HP model:
- wash the data collected from HP's report
- make predictions

In [19]:
class predictor:
    def __init__(self) -> None:
        K1 = 17
        K2 = 1
        K3 = 80
    def setup(self, CPU_core_num, DRAM, SSD, HDD, year):
        return 0
    def print():
        return 0

In [20]:
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("original_data/HP_data.csv")
features = df.columns.values.tolist()
print(features)

['Year', 'Model', 'CPU quant', '#cores per CPU', 'DRAM', 'SSD quant', 'size per SSD', 'Embodied Carbon ', 'TEC', 'Operational Carbon', 'Transportation', 'EOL carbon', 'Total carbon', 'std', 'Mainboard carbon', 'SSD carbon', 'Daughterboard carbon', 'Enclose/Fan/PSU carbon']


extract features for model

In [21]:
features = ['CPU core number', 'DRAM size','Year']
logits = ['total carbon']

SSD_carbon_per_GB = 0.16
fixed_intercept = 1100

x = pd.DataFrame(columns=features,dtype=float)
y = pd.DataFrame(columns=logits,dtype=float)
for idx,row in df.iterrows():
    # if idx==0 or idx==2 or idx == 5 or idx==8:
    if idx in [5,8,2]:
        continue
    if pd.isna(row['#cores per CPU']):#clean entries without critical frature
        continue
    mm,dd,yy = map(int,row["Year"].split('/'))
    year = yy-2000
    if mm==11:
        year += 0.9
    elif mm==4:
        year += 0.3
    else:
        year += mm/12
    CPU_core_number = row['CPU quant']*row['#cores per CPU']
    DRAM_size = row['DRAM']
    x.loc[idx] = [CPU_core_number, DRAM_size, year]

    #here we fix the ssd coefficient by modifying the total carbon before LP
    SSD_size = row['size per SSD']*row['SSD quant']
    y.loc[idx] = row['Embodied Carbon '] - SSD_carbon_per_GB*SSD_size +fixed_intercept

    

print("/////x://///")
print(x)
print("/////y://///")
print(y)

/////x://///
   CPU core number  DRAM size  Year
0             12.0       64.0  22.9
1             32.0      512.0  22.9
3             12.0       64.0  17.9
4             32.0      256.0  17.9
6              8.0       64.0  21.3
7             32.0      512.0  21.3
/////y://///
   total carbon
0        1894.4
1        2462.0
3        1794.4
4        1861.0
6        1878.4
7        2513.0


Linear Regression

In [22]:
from sklearn import  linear_model
from sklearn.metrics import mean_squared_error as MSE
LR= linear_model.LinearRegression(fit_intercept=False, positive=True)
# LR= linear_model.LinearRegression()
model = LR.fit(x,y)

print(model.coef_)
print(model.intercept_)
print("train_MSE:",MSE(y.values,model.predict(x)))
print("train score",model.score(x,y))


# print(model.predict([[1,2,3]]))

[[ 5.0117352   0.95191529 83.08576834]]
0.0
train_MSE: 11546.287834494287
train score 0.8710027855096113
